##  England NHS GP Reviews Sentiment Analysis
source: https://huggingface.co/datasets/janduplessis886/england-nhs-gp-reviews/viewer/default/train

In [2]:
import pandas as pd

df = pd.read_csv('england-nhs-gp-reviews.csv')
df.head()

,ode,surgeryname,url,title,star_rating,comment,visited_date
0,E81050,asplands-medical-centre,https://www.nhs.uk/services/gp-surgery/aspland...,What's changed?,3,Have been with this practice for a number of y...,August 2022
1,E81050,asplands-medical-centre,https://www.nhs.uk/services/gp-surgery/aspland...,Woburn surgery,5,I have been a patient at this practice for man...,July 2022
2,E81050,asplands-medical-centre,https://www.nhs.uk/services/gp-surgery/aspland...,Don't waste your time GPs never available,1,"Visited my gp, over resistant hypertension. Gr...",June 2022
3,E81050,asplands-medical-centre,https://www.nhs.uk/services/gp-surgery/aspland...,Great practice,5,I contacted the surgery by telephone for a non...,June 2022
4,K82064,fishermead-medical-centre,https://www.nhs.uk/services/gp-surgery/fisherm...,Welcoming and supportive,5,I have great respect for the staff at Fisherme...,July 2023


In [2]:
print("Number of rows: ", df.shape[0],"\nNumber of Columns: ", df.shape[1])

Number of rows:  61955 
Number of Columns:  7


#### Check for missing data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61955 entries, 0 to 61954
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ode           61955 non-null  object
 1   surgeryname   61955 non-null  object
 2   url           61955 non-null  object
 3   title         61955 non-null  object
 4   star_rating   61955 non-null  int64 
 5   comment       61955 non-null  object
 6   visited_date  61955 non-null  object
dtypes: int64(1), object(6)
memory usage: 3.3+ MB


No missing data as all 61955 rows are non-null for all columns

### Map Ratings to binary sentiment labels (Positive: 1, Neutral: 0, Negative: -1)

In [4]:
df['sentiment'] = df['star_rating'].apply(lambda x: 1 if x > 3 else 0 if x == 3 else -1)
df.head(10)

,ode,surgeryname,url,title,star_rating,comment,visited_date,sentiment
0,E81050,asplands-medical-centre,https://www.nhs.uk/services/gp-surgery/aspland...,What's changed?,3,Have been with this practice for a number of y...,August 2022,0
1,E81050,asplands-medical-centre,https://www.nhs.uk/services/gp-surgery/aspland...,Woburn surgery,5,I have been a patient at this practice for man...,July 2022,1
2,E81050,asplands-medical-centre,https://www.nhs.uk/services/gp-surgery/aspland...,Don't waste your time GPs never available,1,"Visited my gp, over resistant hypertension. Gr...",June 2022,-1
3,E81050,asplands-medical-centre,https://www.nhs.uk/services/gp-surgery/aspland...,Great practice,5,I contacted the surgery by telephone for a non...,June 2022,1
4,K82064,fishermead-medical-centre,https://www.nhs.uk/services/gp-surgery/fisherm...,Welcoming and supportive,5,I have great respect for the staff at Fisherme...,July 2023,1
5,K82064,fishermead-medical-centre,https://www.nhs.uk/services/gp-surgery/fisherm...,Selective service,1,when I tried to register to go on doctors list...,October 2022,-1
6,K82615,walnut-tree-health-centre,https://www.nhs.uk/services/gp-surgery/walnut-...,Rude staff and no GP appointments ever available,1,Staff are rude and talk over you. Cant get an ...,July 2023,-1
7,K82615,walnut-tree-health-centre,https://www.nhs.uk/services/gp-surgery/walnut-...,Thank you,3,I have been in contact with two members of sta...,July 2023,0
8,K82615,walnut-tree-health-centre,https://www.nhs.uk/services/gp-surgery/walnut-...,unprofessional staff,1,I have been ringing for 2 days trying to sort ...,June 2023,-1
9,K82615,walnut-tree-health-centre,https://www.nhs.uk/services/gp-surgery/walnut-...,This practice is failing,1,Queued outside practice from 7:30 to be told a...,April 2023,-1


#### Check value counts of each sentiment

In [5]:
df['sentiment'].value_counts()

sentiment
 1    34748
-1    23921
 0     3286
Name: count, dtype: int64

3286 (5.3%) of comments are rated as 3 (O encoded). May drop this, dependend on how the model performs

### Features and Labels

In [6]:
features = df['comment']
labels = df['sentiment']

In [7]:
features.head()

0    Have been with this practice for a number of y...
1    I have been a patient at this practice for man...
2    Visited my gp, over resistant hypertension. Gr...
3    I contacted the surgery by telephone for a non...
4    I have great respect for the staff at Fisherme...
Name: comment, dtype: object

In [8]:
labels.head()

0    0
1    1
2   -1
3    1
4    1
Name: sentiment, dtype: int64

### WordNetLemmatizer function

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

import string

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import word_tokenize

def lemmatizer(text):
    lemmatizer = WordNetLemmatizer()
    stopWords = stopwords.words('english')
    #Tokenize
    tokens = word_tokenize(text)
    #Lemmatize
    lemmaToken = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stopWords]
    #Remove punctuations
    lemmaToken = [token for token in lemmaToken if token not in string.punctuation]

    lemmaText = ' '.join(lemmaToken)
    return lemmaText

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kianm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kianm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Lemmatize features

In [10]:
features = features.apply(lemmatizer)

In [11]:
features.head()

0    practice number year always found excellent ho...
1    patient practice many year always found helpfu...
2    visited gp resistant hypertension great appoin...
3    contacted surgery telephone non urgent appoint...
4    great respect staff fishermead medical centre ...
Name: comment, dtype: object

### Train, Test, Split ('comment' column only)

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

### Training the Model using TF-IDF and logistic regression

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import make_pipeline

#Preprocess the text with TFIDF vectorizer
#Train the model using Logistic Regression
model = make_pipeline(TfidfVectorizer(), LogisticRegression(max_iter=1000))
model.fit(x_test, y_test)

#Predict with model on test data
y_pred = model.predict(x_test)

#Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

#Confusion Matrix
confMatrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confMatrix)


Accuracy:  0.9270438221289645
Confusion Matrix:
 [[4613    0  143]
 [ 423   49  184]
 [ 154    0 6825]]


Low score for 0 class (neutral, rating 3). Data for 0 class may be too little result in such a low score. i.e. not enough data to train.

Rating 3 has only 5.3% of data.
Drop ratings that are 3 as it is too little data to be meaningful and poor performance.

### Drop rows that have ratings that are 3

In [14]:
df_drop0 = df[df['sentiment']!=0]
df_drop0['sentiment'].value_counts()

sentiment
 1    34748
-1    23921
Name: count, dtype: int64

### Features and labels without sentiments 0

In [15]:
featuresNoZero = df_drop0['comment']
labelsNoZero = df_drop0['sentiment']

In [16]:
featuresNoZero.head()

1    I have been a patient at this practice for man...
2    Visited my gp, over resistant hypertension. Gr...
3    I contacted the surgery by telephone for a non...
4    I have great respect for the staff at Fisherme...
5    when I tried to register to go on doctors list...
Name: comment, dtype: object

In [17]:
labelsNoZero.head()

1    1
2   -1
3    1
4    1
5   -1
Name: sentiment, dtype: int64

#### Lemmatize featureNoZero

In [18]:
featuresNoZero= featuresNoZero.apply(lemmatizer)

In [19]:
featuresNoZero.head()

1    patient practice many year always found helpfu...
2    visited gp resistant hypertension great appoin...
3    contacted surgery telephone non urgent appoint...
4    great respect staff fishermead medical centre ...
5    tried register go doctor list told catchment a...
Name: comment, dtype: object

### Train, Test, Split features and labels(no zero)

In [20]:
x_trainN, x_testN, y_trainN, y_testN = train_test_split(featuresNoZero, labelsNoZero, test_size=0.2, random_state=42)

### Train the model

In [21]:
#TfidfVectorizer process and train using Logistic Regression
modelN = make_pipeline(TfidfVectorizer(),LogisticRegression(max_iter=1000))

#Train the model
modelN.fit(x_trainN, y_trainN)

#Predict with model on test data
y_predN = modelN.predict(x_testN)

#Evaluate model performance
accuracyN = accuracy_score(y_testN, y_predN)
print("Accuracy: ", accuracyN)

#Confusion Matrix
confMatrixN = confusion_matrix(y_testN, y_predN)
print("Confusion Matrix:\n", confMatrixN)


Accuracy:  0.9590080109084711
Confusion Matrix:
 [[4473  237]
 [ 244 6780]]


### Export the model

In [22]:
import pickle
'''
with open('commentSentiment.pkl', 'wb') as file:
    pickle.dump(modelN, file)
'''

"\nwith open('commentSentiment.pkl', 'wb') as file:\n    pickle.dump(modelN, file)\n"

### Load the model

In [23]:
loaded_model = pickle.load(open('commentSentiment.pkl', 'rb'))

### Getting Keywords from comments

In [24]:
featuresClean = features.apply(lemmatizer)
featuresClean.head(10)

0    practice number year always found excellent ho...
1    patient practice many year always found helpfu...
2    visited gp resistant hypertension great appoin...
3    contacted surgery telephone non urgent appoint...
4    great respect staff fishermead medical centre ...
5    tried register go doctor list told catchment a...
6    staff rude talk cant get appointment love mone...
7    contact two member staff practice last week on...
8    ringing 2 day trying sort medication due 2 day...
9    queued outside practice 7:30 told appointment ...
Name: comment, dtype: object

### Text Featurization

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

# convert to NumPy array
tfidf_vecs_model = tfidf.fit(featuresClean)



### Export tfidfVect Mode

In [28]:
import pickle

with open('tfidfVec.pkl', 'wb') as file:
    pickle.dump(tfidf_vecs_model, file)


In [1]:
import pickle
loaded_model =  pickle.load(open('tfidfVec.pkl', 'rb'))

### TODO:
1. Code a flask server to serve the model
2. Test the model with flask server
3. Host on AWS??